In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import time
from helper_functions import *
from all_config import africa_countries, latam_countries, asia_countries, europe_countries


Countries


In [2]:
# update this to have validation test functions for
# each single map,
# ggit map
# multi tracker map
# gas only multi tracker map
# for europe remember ggit hy is the one not ggit

# Then run for all remaining maps needed: AET (multi), LATAM (multi), agt (gas only), egt (gas only), ggit (special), gmet (single)
# Verify that new admin panel aligns with current code config for geo and fuel filters
# Use date last researched and file title to verify version

In [3]:
# # # one off source file update # TODO
# key = '1WRRFRuR9mWxZpko-VMYH5xm0LzRX_qM7W73nGGi4Jmg'
# tabs = ['Data', 'Below Threshold']
# tracker='bio' 
# dfs = []
# # # maybe download and add to pre test, if pass then upload to data team folder and the source folder, use to create the maps
# for tab in tabs:
#     gsheets = gspread_creds.open_by_key(key)
#     spreadsheet = gsheets.worksheet(tab)
#     df = pd.DataFrame(spreadsheet.get_all_records(expected_headers=[]))
#     dfs += [df]
# df = pd.concat(dfs).reset_index(drop=True)
# print(df['Capacity (MW)'].sum())
# # print((df.groupby('Project Name')['Capacity (MW)']).max()) # unit max or do unit summed per project max 
# print(len(df['Project Name']))
# print(len(df['Unit Name']))
# print(df['Status'].value_counts())

In [4]:
# df.info()
# # df = df.rename(columns=renaming_cols_dict['GBPT'])
# # print(df['areas'].value_counts())
# # print(renaming_cols_dict['GBPT'])
# # df.info()
# for area in df['Country/Area']:
#     if area in full_country_list:
#         pass
#     else:
#         print(f'This area is not in the list: {area}')
# print('Done checking countries')


In [5]:

# little configurations
testing_source = True
filter_country = False
single_map_file = False

<!-- Map: asia
Total number of DataFrames: 2
Tracker counts: Counter({'GOGPT': 4858, 'GGIT': 1645, 'GOGET': 907, 'GGIT-lng': 481})
----------------------------------------
Map: europe
Total number of DataFrames: 2
Tracker counts: Counter({'GOGPT': 2675, 'GOGET': 1202, 'GGIT': 1062, 'GGIT-lng': 168}) -->

In [6]:


# NOTE: CONFIGURATIONS TO BE CHANGED AS NEEDED PER TESTER AND CYCLE
map_to_test = 'ggit_hy' 

testing_source_path = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/'
testing_final_path = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/final/'
testing_errors_path = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/'
# use the same version of each tracker as was used for current version of Africa Energy map
coal_mines_file = 'Global-Coal-Mine-Tracker-April-2024 DATA TEAM COPY.xlsx' #'Global-Coal-Mine-Tracker-April-2024.xlsx'
gogpt_file = 'Global Oil and Gas Plant Tracker (GOGPT) - August 2024 DRAFT v2 DATA TEAM COPY.xlsx' #'Global Oil and Gas Plant Tracker (GOGPT) - February 2024 v4.xlsx'
gcpt_file = 'Global-Coal-Plant-Tracker-July-2024 DATA TEAM COPY.xlsx' #'Global-Coal-Plant-Tracker-July-2024.xlsx'
hydro_file = 'Global-Hydropower-Tracker-April-2024 DATA TEAM COPY.xlsx' #'Global-Hydropower-Tracker-April-2024.xlsx'
bio_file = 'Global-Bioenergy-Power-Tracker-GBPT-V2 DATA TEAM COPY.xlsx' #'Global-Bioenergy-Power-Tracker-2023-11.xlsx'
wind_file = 'Global-Wind-Power-Tracker-June-2024 DATA TEAM COPY.xlsx' #'Global-Wind-Power-Tracker-June-2024.xlsx'
goget_file = 'Global Oil and Gas Extraction Tracker - 2024-03-08_1205 DATA TEAM COPY.xlsx'
goget_formatted ='GOGET Earthgenome file 2024-07-03_200828.xlsx'
ggit_file = 'GEM-GGIT-Gas-Pipelines-2023-12 copy.geojson'
ggit_lng_file = 'GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson'
ggit_hy_file = 'Hydrogen Pipelines for EGT briefing in Feb 2024.xlsx'
nuclear_file = 'Global-Nuclear-Power-Tracker-July-2024 DATA TEAM COPY.xlsx'
coal_terminals_file = 'Global-Coal-Terminals-Tracker-December-2023 DATA TEAM COPY.xlsx'
goit_file = 'GEM-GOIT-Oil-NGL-Pipelines-2024-06 copy.geojson'
geothermal_file = 'Geothermal-Power-Tracker-May-2024 DATA TEAM COPY.xlsx'
solar_file = 'Global-Solar-Power-Tracker-June-2024 DATA TEAM COPY.xlsx'
coal_term_file = "Global-Coal-Terminals-Tracker-December-2023 DATA TEAM COPY.xlsx"

# create all source dfs
dfs = []
for sheet_name  in ['Main data', 'Production and reserves']: # TODO add in Scotts code, main data is main data, pull in prod as needed or use formatted for immediate test
    df = pd.read_excel(testing_source_path + goget_file, sheet_name=sheet_name,
        dtype=dtype_spec)
    dfs += [df]
goget = pd.concat(dfs).reset_index(drop=True)

goget_formatted = pd.read_excel(testing_source_path + goget_formatted, sheet_name='Data')
ggit_lng = gpd.read_file(testing_source_path + ggit_lng_file)
ggit_hy = pd.read_excel(testing_source_path + ggit_hy_file, sheet_name = 'Gas Pipelines')
ggit = gpd.read_file(testing_source_path + ggit_file)
goit = gpd.read_file(testing_source_path + goit_file)
nuclear = pd.read_excel(testing_source_path + nuclear_file, sheet_name='Data',
        dtype=dtype_spec)
coal_term = pd.read_excel(testing_source_path + coal_terminals_file, sheet_name='Coal Terminals',
        dtype=dtype_spec)

dfs= []
for sheet_name in ['Data', 'Below Threshold']:
    df = pd.read_excel(
    testing_source_path + wind_file,
    sheet_name = sheet_name,
    dtype=dtype_spec
)
    dfs += [df]
geo = pd.concat(dfs).reset_index(drop=True)


dfs = []
for sheet_name in['20 MW+', '1-20 MW']:
    df = pd.read_excel(testing_source_path + solar_file, sheet_name=sheet_name,
        dtype=dtype_spec)
    dfs += [df]
solar = pd.concat(dfs).reset_index(drop=True) 

dfs = []
for sheet_name in ['Gas & Oil units', 'Sub-threshold units']:
    df = pd.read_excel(
        testing_source_path + gogpt_file, 
        sheet_name = sheet_name,
        dtype=dtype_spec)
    dfs += [df]
gogpt = pd.concat(dfs).reset_index(drop=True) 

gcpt = pd.read_excel(
    testing_source_path + gcpt_file,
    sheet_name = 'Units',
        dtype=dtype_spec
)
dfs = [] # initialize
for sheet_name in ['Global Coal Mine Tracker (Non-C', 'Global Coal Mine Tracker (Close']:
    df = pd.read_excel(
        testing_source_path +
        coal_mines_file,
        sheet_name = sheet_name,
        dtype=dtype_spec
        
    )
    if sheet_name == 'Global Coal Mine Tracker (Close':
        if 'Status' not in df.columns.to_list():
            df['Status'] = 'Retired'
    dfs += [df]
coal_mines = pd.concat(dfs).reset_index(drop=True)

hydro = pd.read_excel(
    testing_source_path + hydro_file,
    sheet_name = 'Data',
        dtype=dtype_spec
)

dfs= []
for sheet_name in ['Data', 'Below Threshold']:
    df = pd.read_excel(
    testing_source_path + wind_file,
    sheet_name = sheet_name,
    dtype=dtype_spec
)
    dfs += [df]
wind = pd.concat(dfs).reset_index(drop=True) 

dfs= []
for sheet_name in ['Data', 'Below Threshold']:
    df = pd.read_excel(
    testing_source_path + wind_file,
    sheet_name = sheet_name,
    dtype=dtype_spec
)
    dfs += [df]
bio = pd.concat(dfs).reset_index(drop=True)


coal_terminals = pd.read_excel(testing_source_path + coal_term_file,
                               sheet_name = 'Coal Terminals',
                               dtype=dtype_spec)




FileNotFoundError: [Errno 2] No such file or directory: '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/Hydrogen Pipelines for EGT briefing in Feb 2024.xlsx'

In [164]:
ggit_hy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3790 entries, 0 to 3789
Columns: 131 entries, PipelineNetworkGrouping to EGTImport
dtypes: float64(21), int64(1), object(109)
memory usage: 3.8+ MB


In [210]:
mth = ggit_hy[ggit_hy['Fuel']=='Gas'] # 3653

mth['country_to_check'] = mth['Countries'].str.split(',')

print(mth[mth['PipelineName'] == 'Malta-Italy Gas Pipeline']['country_to_check'])
# filter the df on the country column to see if any of the countries in that list is in the needed geo
# split out countries, or start and end country, if either in europe coutnries 
# mth_eu # 700

keep = []
for row in mth.index:
    country_list = mth.loc[row, 'country_to_check']
    # print(country_list)
    for item in country_list:
        # print(item)
        if item in europe_countries:
            keep.append(row)
# print(europe_countries)
# filter with keep after deduplicating 
keep = list(set(keep))
# only keep rows in mth that are in keep
mth_eur = mth.loc[keep]
# print(mth_eur[mth_eur['PipelineName'] == 'Malta-Italy Gas Pipeline'])
# mth[mth['PipelineName'] == 'Malta-Italy Gas Pipeline']
mth_eu_5 = mth_eur[mth_eur['PCI5'] == 'yes']
print(len(mth_eu_5)) # 9
mth_eu_6 = mth_eur[mth_eur['PCI6'] == 'yes'] # should be 40
print(len(mth_eu_6))

781    [Italy,  Malta]
Name: country_to_check, dtype: object
19
3


/var/folders/70/9xc3s63n0j9crf3st7fc61cm0000gn/T/ipykernel_29836/3085737567.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mth['country_to_check'] = mth['Countries'].str.split(',')


In [212]:
hy = ggit_hy[ggit_hy['Fuel']=='Hydrogen'] # 114 
# hy_eu = hy[hy['Region']=='Europe']
# for col in hy.columns.to_list():
#     print(col)
# if any of the countries in the country column list is in the needed geo list then keep it if none then filter out
# for sep in ',;-':
#     # I want to break up any multiple countries into a list of countries
#     # then check if any of those are in the needed_geo list
#     hy['country_to_check'] = hy['Countries'].str.strip().str.split(sep) 
hy['country_to_check'] = hy['Countries'].str.split(',')

keep = []
for row in hy.index:
    country_list = hy.loc[row, 'country_to_check']
    # print(country_list)
    for item in country_list:
        item = item.strip()
        # print(item)

        if item in europe_countries:
            keep.append(row)
# print(europe_countries)
# filter with keep after deduplicating 
keep = list(set(keep))
# only keep rows in mth that are in keep
hy_eur = hy.loc[keep]
# filter the df on the country column to see if any of the countries in that list is in the needed geo
# hy_eu = hy[hy['country_to_check'].apply(lambda x: check_list(x, europe_countries))]
# split out countries, or start and end country, if either in europe coutnries 
# hy_eur

hy_eur_5 = hy_eur[hy_eur['PCI5'] == 'yes']
print(len(hy_eur_5)) # 9
hy_eur_6 = hy_eur[hy_eur['PCI6'] == 'yes'] # should be 40
print(len(hy_eur_6))

0
37


/var/folders/70/9xc3s63n0j9crf3st7fc61cm0000gn/T/ipykernel_29836/3968914182.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hy['country_to_check'] = hy['Countries'].str.split(',')


In [214]:
hy_eu.columns # 86 in EU

Index(['PipelineNetworkGrouping', 'PipelineName', 'SegmentName', 'Wiki',
       'ProjectID', 'Researcher', 'LastUpdated', 'Fuel', 'Fuel [ref]',
       'PipelineType',
       ...
       'AssociatedWithLNGExports', 'AssociatedLNGTerminal', 'DraftPCI6List',
       'PCI6ProjectCode', 'PipelineDirectionality', 'QCCOwner',
       'QCCOwner [ref]', 'ImpactedByRussiaUkraineInvasion', 'EGTImport',
       'country_to_check'],
      dtype='object', length=132)

In [224]:
ggit_hy_mth = pd.concat([hy_eur, mth_eur])

ggit_hy_mth_ids = set(ggit_hy_mth['ProjectID'])

In [242]:
ggit_new = pd.concat([ggit_hy_mth, ggit])

In [246]:
ggit_new[ggit_new['PCI6']=='yes']

,PipelineNetworkGrouping,PipelineName,SegmentName,Wiki,ProjectID,Researcher,LastUpdated,Fuel,Fuel [ref],PipelineType,...,AssociatedLNGTerminal,DraftPCI6List,PCI6ProjectCode,PipelineDirectionality,QCCOwner,QCCOwner [ref],ImpactedByRussiaUkraineInvasion,EGTImport,country_to_check,geometry
3152,NaN,AquaDuctus Hydrogen Pipeline,Phase 1 (Helgoland–Aquaprimus II),https://www.gem.wiki/AquaDuctus_Hydrogen_Pipeline,P5110,HH,2023-01-06,Hydrogen,https://www.entsog.eu/tyndp#entsog-ten-year-ne...,transmission,...,NaN,yes,HYD-N-991,NaN,NaN,NaN,NaN,NaN,[Germany],None
3153,NaN,AquaDuctus Hydrogen Pipeline,Phase 2,https://www.gem.wiki/AquaDuctus_Hydrogen_Pipeline,P5111,HH,2023-01-07,Hydrogen,https://www.entsog.eu/tyndp#entsog-ten-year-ne...,transmission,...,NaN,yes,HYD-N-991,NaN,NaN,NaN,NaN,NaN,[Germany],None
3154,NaN,AquaDuctus Hydrogen Pipeline,Phase 3,https://www.gem.wiki/AquaDuctus_Hydrogen_Pipeline,P5112,HH,2023-01-08,Hydrogen,https://www.entsog.eu/tyndp#entsog-ten-year-ne...,transmission,...,NaN,yes,HYD-N-991,NaN,NaN,NaN,NaN,NaN,[Germany],None
3160,NaN,H2Med Pipeline,BarMar Pipeline,https://www.gem.wiki/H2Med_Pipeline,P5119,HH,2023-01-11,Hydrogen,https://www.grtgaz.com/en/our-energy-transitio...,transmission,...,NaN,yes,HYD-N-1153; HYD-N-1151; HYD-N-819,NaN,NaN,NaN,NaN,NaN,"[Spain, France]",None
3740,NaN,Portugal Hydrogen Pipeline System,"Phase 1 (EHB 2030): Cantanhede–Mangualde, Celo...",NaN,P6201,BL,2024-01-26,Hydrogen,https://h2medproject.com/building-the-european...,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Portugal],None
3742,NaN,HY-FEN Pipeline,NaN,NaN,P6203,BL,2024-01-26,Hydrogen,https://www.grtgaz.com/en/our-energy-transitio...,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[France, Germany]",None
3743,NaN,H2ercules Pipeline,2026 Network (GET H2),NaN,P6204,BL,2024-01-30,Hydrogen,https://www.h2ercules.com/en,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany],None
3744,NaN,H2ercules Pipeline,2028 Network,NaN,P6205,BL,2024-01-30,Hydrogen,https://www.h2ercules.com/en,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany],None
3745,NaN,H2ercules Pipeline,2030 Network,NaN,P6206,BL,2024-01-30,Hydrogen,https://www.h2ercules.com/en,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany],None
3747,NaN,Belgium Hydrogen Pipeline System,2040 Phase,NaN,P6208,BL,2024-02-01,Hydrogen,https://northsearegion.eu/media/23250/5-fluxys...,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Belgium],None


In [247]:
ggit_new = ggit_new['ProjectID'].drop_duplicates()

In [248]:
len(ggit_new)

3734

In [223]:
ggit_lng['TerminalID']
ggit_ids = set(ggit['ProjectID'])

In [226]:
notin = ggit_hy_mth[~ggit_hy_mth['ProjectID'].isin(ggit['ProjectID'])]

In [233]:
isin = ggit_hy_mth[ggit_hy_mth['ProjectID'].isin(ggit['ProjectID'])]

In [241]:
isin[isin['PCI5'] == 'yes']

,PipelineNetworkGrouping,PipelineName,SegmentName,Wiki,ProjectID,Researcher,LastUpdated,Fuel,Fuel [ref],PipelineType,...,AssociatedWithLNGExports,AssociatedLNGTerminal,DraftPCI6List,PCI6ProjectCode,PipelineDirectionality,QCCOwner,QCCOwner [ref],ImpactedByRussiaUkraineInvasion,EGTImport,country_to_check
308,Baltic Pipe Project,Baltic Pipe Project,Main Segment,https://www.gem.wiki/Baltic_Pipe_Project,P0684,BL,2023-01-26,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Norway, Denmark, Sweden, Poland]"
320,NaN,Gas Interconnector Greece–Bulgaria (IGB),NaN,https://www.gem.wiki/Gas_Interconnector_Greece...,P0696,HH,2023-08-08,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes - explicit,NaN,"[Greece, Bulgaria]"
416,NaN,EastMed Gas Pipeline,NaN,https://www.gem.wiki/EastMed_Gas_Pipeline,P0827,BL,2024-02-06,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes - explicit,yes?,"[Israel, Cyprus, Greece]"
781,NaN,Malta-Italy Gas Pipeline,NaN,https://www.gem.wiki/Malta-Italy_Gas_Pipeline,P1327,BL,2024-02-06,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Italy, Malta]"
787,NaN,Bulgaria-Serbia Interconnector Gas Pipeline,NaN,https://www.gem.wiki/Bulgaria_Serbia_Interconn...,P1336,RR,2024-02-02,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Bulgaria, Serbia]"
790,NaN,Slovakia-Hungary Interconnector Gas Pipeline,NaN,https://www.gem.wiki/Slovakia-Hungary_Intercon...,P1341,HH,2022-08-30,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Hungary]
791,NaN,Croatia-Slovenia Interconnector Gas Pipeline,NaN,https://www.gem.wiki/Croatia-Slovenia-Austria_...,P1342,HH,2023-09-24,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Croatia, Slovenia]"
794,NaN,Poseidon Gas Pipeline,NaN,https://www.gem.wiki/Poseidon_Gas_Pipeline,P1345,HH,2023-08-08,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Greece, Italy]"
947,NaN,Matagiola-Massafra Gas Pipeline,NaN,https://www.gem.wiki/Matagiola-Massafra_Gas_Pi...,P1691,HH,2023-08-07,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Italy]
956,NaN,Black Sea Shore–Podișor Gas Pipeline,NaN,https://www.gem.wiki/Black_Sea_Shore%E2%80%93P...,P1702,BL,2023-07-06,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Romania]


In [239]:
set(isin['Fuel'].to_list())
isin[isin['WKTFormat'] != '--'] # 764 hydrogen and methane with geo and have overlap


,PipelineNetworkGrouping,PipelineName,SegmentName,Wiki,ProjectID,Researcher,LastUpdated,Fuel,Fuel [ref],PipelineType,...,AssociatedWithLNGExports,AssociatedLNGTerminal,DraftPCI6List,PCI6ProjectCode,PipelineDirectionality,QCCOwner,QCCOwner [ref],ImpactedByRussiaUkraineInvasion,EGTImport,country_to_check
3378,NaN,RHYn Pipeline,Fessenheim–Ottmarsheim–Mulhouse,https://www.gem.wiki/RHYn_Pipeline,P5663,BL,2024-02-17,Hydrogen,https://www.grtgaz.com/en/medias/press-release...,transmission,...,NaN,NaN,yes,HYD-N-969,NaN,NaN,NaN,NaN,NaN,[France]
2052,NaN,Hungary-Austria Interconnector Gas Pipeline,Austrian segment,https://www.gem.wiki/Hungary-Austria_Interconn...,P3967,HH,2022-07-19,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Austria]
2053,NaN,Hungary-Austria Interconnector Gas Pipeline,Hungarian Segment,https://www.gem.wiki/Hungary-Austria_Interconn...,P3968,HH,2023-09-05,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Austria, Hungary]"
2361,NaN,Pordenone-Istrana Gas Pipeline,NaN,https://www.gem.wiki/Pordenone-Istrana_Gas_Pip...,P4290,HH,2022-08-22,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Italy]
2055,NaN,Austria–Slovenia Interconnector Gas Pipeline,NaN,https://www.gem.wiki/Austria%E2%80%93Slovenia_...,P3970,HH,2022-07-19,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Austria, Slovenia]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,NaN,Pancevo-Horgos Gas Pipeline,Pancevo-Elemir,https://www.gem.wiki/Pancevo-Horgos_Gas_Pipeline,P4369,HH,2022-09-16,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Serbia]
2441,NaN,Pojate-Leskovac Gas Pipeline,NaN,https://www.gem.wiki/Pojate-Leskovac_Gas_Pipeline,P4371,HH,2022-09-16,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Serbia]
2044,NaN,Barents Stream Pipeline,NaN,https://www.gem.wiki/Barents_Stream_Pipeline,P3958,HH,2022-07-18,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Norway]
2045,NaN,Cyprus-Turkey Gas Pipeline,Vasilikos-Girne,https://www.gem.wiki/Cyprus-Turkey_Gas_Pipeline,P3959,HH,2022-07-18,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Cyprus]


In [227]:
notin

,PipelineNetworkGrouping,PipelineName,SegmentName,Wiki,ProjectID,Researcher,LastUpdated,Fuel,Fuel [ref],PipelineType,...,AssociatedWithLNGExports,AssociatedLNGTerminal,DraftPCI6List,PCI6ProjectCode,PipelineDirectionality,QCCOwner,QCCOwner [ref],ImpactedByRussiaUkraineInvasion,EGTImport,country_to_check
3148,NaN,Hamburg Hydrogen Network Pipeline,NaN,NaN,P5104,HH,2023-01-05,Hydrogen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany]
3149,NaN,Green Octopus Mitteldeutschland,New Segments,https://www.gem.wiki/Green_Octopus_Mitteldeuts...,P5105,HH,2023-01-06,Hydrogen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany]
3150,NaN,Get H2 Pipeline,Lingen-Gelsenkirchen,https://www.gem.wiki/Get_H2_Pipeline,P5107,HH,2023-01-08,Hydrogen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany]
3151,NaN,Get H2 Pipeline,Phase 2,https://www.gem.wiki/Get_H2_Pipeline,P5108,HH,2023-01-09,Hydrogen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany]
3152,NaN,AquaDuctus Hydrogen Pipeline,Phase 1 (Helgoland–Aquaprimus II),https://www.gem.wiki/AquaDuctus_Hydrogen_Pipeline,P5110,HH,2023-01-06,Hydrogen,https://www.entsog.eu/tyndp#entsog-ten-year-ne...,transmission,...,NaN,NaN,yes,HYD-N-991,NaN,NaN,NaN,NaN,NaN,[Germany]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735,NaN,Mukran–Lubmin Connecting Pipeline,NaN,NaN,P6039,BL,2023-09-21,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Germany]
3629,NaN,Tocuz-Cainari-Mereni Gas Pipeline,NaN,NaN,P5933,HH,2023-09-07,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Moldova]
3630,NaN,Olishcani-Saharna Gas Pipeline,NaN,NaN,P5934,HH,2023-09-07,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Moldova]
3631,NaN,Odessa-Chisinau Gas Pipeline,NaN,NaN,P5935,HH,2023-09-07,Gas,NaN,transmission,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Moldova, Ukraine]"


In [136]:


for file in os.listdir(testing_final_path): 
    if (map_to_test in file) & file.endswith(".geojson") and iso_today_date in file:
        final_map = gpd.read_file(testing_final_path + file)
        print(file)

    elif (map_to_test in file) & file.endswith(".xlsx") & (new_release_date in file) & ('about' in file):
        # final_data = pd.ExcelFile(testing_final_path + file)
        # Read all sheets into a dictionary
        final_data_dict_with_about = pd.read_excel(testing_final_path + file, sheet_name=None)
        print(file)
 
    elif (map_to_test in file) & file.endswith(".xlsx") & (new_release_date in file):
        # final_data = pd.ExcelFile(testing_final_path + file)
        # Read all sheets into a dictionary
        final_data_dict = pd.read_excel(testing_final_path + file, sheet_name=None) 
        print(file)

    # elif ('Integrated' in file) & (file.endswith(".xlsx")):
    #     gipt_james_version = pd.read_excel(testing_final_path + file, sheet_name='Power facilities')
    #     print(file)
    # TODO add in the summary data files when we generate them
print(f"We will be testing for map {map_to_test}.")
print(f"Make sure all local source files have been updated for this cycle.")


ggit_2024-10-08.geojson
We will be testing for map ggit.
Make sure all local source files have been updated for this cycle.


In [137]:
final_map.columns

Index(['id', 'url', 'name', 'unit-name', 'status', 'areas', 'owner', 'parent',
       'capacity', 'start-year', 'region', 'subnat', 'Latitude', 'Longitude',
       'tracker-acro', 'tracker-custom', 'subnat2', 'region2',
       'scaling-capacity', 'capacity-table', 'capacity-details',
       'status-legend', 'areas-subnat-sat-display', 'tracker-display',
       'geometry'],
      dtype='object')

TEST SOURCE FILE FOR MISSING OR MISFORMATTED COLUMNS

In [138]:
if testing_source == True:
    print(
        'create good tests for countries'
    )
    # this could also be on final
    # to avoid region fiilter issue
    # test all countries are legit from areas
    # mauritania-senegal snuck through
    # canary islands too
    # maybe note the  number of mutli country per tracker

In [139]:
if testing_source == True:
    
    misformatted_cols_check = ['geometry', 'Latitude', 'Longitude', 'status', 'url', 'route', 'areas'] #'status', 'url', 'areas',
    misformatted = {}
    # list_of_trackers = [(coal_terminals, 'GCTT'), (coal_mines, 'GCMT'), (gcpt,'GCPT'), (gogpt, 'GOGPT'), (hydro, 'GHPT'), (wind,'GWPT'), (bio,'GBPT'),
    #                     (solar, 'GSPT'), (geo, 'GGPT'), (nuclear, 'GNPT'), (goget_formatted, 'GOGET'), (ggit_lng, 'GGIT-lng'), (ggit, 'GGIT'), (goit, 'GOIT')]
    list_of_trackers = [(df, 'GBPT')]
    file_name = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/output[misformmatted essential data from source]{new_release_date}.xlsx'
    print(file_name)
    with pd.ExcelWriter(file_name) as writer:
        for tracker_tup in list_of_trackers:
            df = tracker_tup[0]
            tracker = tracker_tup[1]
            print(tracker)
            renaming_sel = renaming_cols_dict[tracker]
            # print(renaming_sel)
            df = df.rename(columns=renaming_sel) 
            
            df['error messages'] = '' # initialize

                
            for col in misformatted_cols_check:
                print(col)
                if col in df.columns:
                    ser = df[col].replace('', np.nan)
                    try:
                        ser_float = ser.astype(float)
                    except:
                        for row in ser.index:
                            val = ser.at[row]
                            try:
                                val_float = float(val)
                            except:
                                # print("Error!" + f" val couldn't be converted to float: {val}")
                                df.at[row, 'error messages'] += "Error!" + f" In col {col}, val couldn't be converted to float: '{val}'; "

    df = df[df['error messages']!='']
    df['error messages'] = df['error messages'].str.strip('; ')
    df.to_excel(writer, sheet_name='error messages', index=False)



In [140]:
if testing_source == True:

    # RUN ON SOURCE DATA ONLY
    # add a test to compare region and country list inconsistency

    # find source data where key info is missing
    # status
    # geometry or lat lng
    # include countries that are multiple so not in latam 
    missing_cols_check = ['geometry', 'Latitude', 'Longitude'] #'status', 'url', 'name', 'id', 'areas', 
    missing = {}
    misformatted = {}
    inconsistent = {} # create a check to compare region to country 

    list_of_trackers = [(coal_terminals, 'GCTT'), (coal_mines, 'GCMT'), (gcpt,'GCPT'), (gogpt, 'GOGPT'), (hydro, 'GHPT'), (wind,'GWPT'), (bio,'GBPT'),
                        (solar, 'GSPT'), (geo, 'GGPT'), (nuclear, 'GNPT'), (goget_formatted, 'GOGET'), (ggit_lng, 'GGIT-lng'), (ggit, 'GGIT'), (goit, 'GOIT')]
    # list_of_trackers = [(coal_terminals, 'GCTT Coal Terminals'), (coal_mines, 'coal_mines'), (gcpt,'gcpt')]
    file_name = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/output-missing-misformatted-essential-map-{new_release_date}{iso_today_date}.xlsx' # trackers_to_update

    def check_missing_misformatted(df):
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer: 

            for tracker_tup in list_of_trackers:
                df = tracker_tup[0]
                tracker = tracker_tup[1]
                renaming_sel = renaming_cols_dict[tracker]
                df = df.rename(columns=renaming_sel) 
                df['error messages'] = '' # initialize
                
    # missing test

                for col in missing_cols_check:
                    df['error messages'] = '' # reset for each column so error messages relevant in tab

                    print(col)
                    if col in df.columns:
                        print(f"Data type of column '{col}': {df[col].dtype}")
                        print(f"Number of missing values in column '{col}': {df[col].isna().sum()}")
                        df_missing_na = df[col].isna()
                        # print(len(df))
                        df_missing_nan = df.loc[df_missing_na]
                        # print(len(df_missing_nan))
                        df_missing_blank = df[df[col]=='']
                        df_missing = pd.concat([df_missing_nan, df_missing_blank], axis=0)
                        df_missing = df_missing.drop_duplicates(subset=['id'])
                        if len(df_missing) > 0:
                            missing[f'{tracker}-{col}'] = df_missing
                            df_missing.to_excel(writer, sheet_name=f'{tracker}-{col}-missing', index=False)
    # misformatted test
                        ser = df[col].replace('', np.nan)
                        try:
                            ser_float = ser.astype(float)
                        except:
                            for row in ser.index:
                                val = ser.at[row]
                                try:
                                    val_float = float(val)
                                except:
                                    df.at[row, 'error messages'] += "Error!" + f" In col {col}, val couldn't be converted to float: '{val}'; "
                        
                        
                        df_misformatted = df[df['error messages']!=''] # means there is a col with a misformatted row 
                        df_misformatted['error messages'] = df_misformatted['error messages'].str.strip('; ')
                        if len(df_misformatted) > 0:
                            df_misformatted.to_excel(writer, sheet_name=f'{tracker}-{col}-misformatted', index=False)            
                    else:
                        print(f'{col} not in {tracker}')
                # print(missing)

    check_missing_misformatted(df)



In [141]:
final_data_dict_with_about

{'About Europe Energy Tracker':                                                     0
 0                                               About
 1                                                 NaN
 2   The Europe Gas Tracker (EGT) is a project of G...
 3                                                 NaN
 4                                           Countries
 5                                                 NaN
 6   The Europe Gas Tracker includes data from all ...
 7                                                 NaN
 8   The full list of countries in the Tracker is a...
 9                                                 NaN
 10  Creative Commons License\n\nCopyright © Global...
 11                                                NaN
 12       Data are derived from GEM’s global trackers:
 13  Gas plants from the Global Gas Plant Tracker (...
 14  Methane gas pipelines from the Global Gas Infr...
 15  LNG terminals from the Global Gas Infrastructu...
 16  Gas extraction sites from the

In [142]:
final_data_dict

{'Gas Pipelines':     ProjectID Fuel    StartCountry       EndCountry       Countries  \
 0       P0686  Gas  United Kingdom   United Kingdom  United Kingdom   
 1       P0688  Gas         Germany          Germany         Germany   
 2       P0691  Gas  United Kingdom   United Kingdom  United Kingdom   
 3       P0693  Gas  United Kingdom   United Kingdom  United Kingdom   
 4       P0694  Gas  United Kingdom   United Kingdom  United Kingdom   
 ..        ...  ...             ...              ...             ...   
 675     P5899  Gas        Bulgaria         Bulgaria        Bulgaria   
 676     P5900  Gas        Bulgaria         Bulgaria        Bulgaria   
 677     P5902  Gas  Czech Republic  Czech Republiic  Czech Republic   
 678     P5918  Gas  Czech Republic  Czech Republiic  Czech Republic   
 679     P5986  Gas          Poland           Poland          Poland   
 
                                                   Wiki  \
 0    https://www.gem.wiki/Central_Area_Transmission...   

In [143]:
final_data_dict_with_about

{'About Europe Energy Tracker':                                                     0
 0                                               About
 1                                                 NaN
 2   The Europe Gas Tracker (EGT) is a project of G...
 3                                                 NaN
 4                                           Countries
 5                                                 NaN
 6   The Europe Gas Tracker includes data from all ...
 7                                                 NaN
 8   The full list of countries in the Tracker is a...
 9                                                 NaN
 10  Creative Commons License\n\nCopyright © Global...
 11                                                NaN
 12       Data are derived from GEM’s global trackers:
 13  Gas plants from the Global Gas Plant Tracker (...
 14  Methane gas pipelines from the Global Gas Infr...
 15  LNG terminals from the Global Gas Infrastructu...
 16  Gas extraction sites from the

In [144]:
final_map

,id,url,name,unit-name,status,areas,owner,parent,capacity,start-year,...,tracker-custom,subnat2,region2,scaling-capacity,capacity-table,capacity-details,status-legend,areas-subnat-sat-display,tracker-display,geometry
0,T020501,https://www.gem.wiki/Fourchon_LNG_Terminal,Fourchon LNG Terminal,Phase 1,cancelled,United States,Fourchon LNG LLC [100.00%],Energy World [100.00%],2,2021,...,GGIT - export,,,87744.60,2.0 mtpa of LNG,,cancelled,"Louisiana,United States",LNG export terminal,POINT (-90.19444 29.10583)
1,T020502,https://www.gem.wiki/Fourchon_LNG_Terminal,Fourchon LNG Terminal,Phase 2,cancelled,United States,Fourchon LNG LLC [100.00%],Energy World [100.00%],3,2023,...,GGIT - export,,,131616.90,3.0 mtpa of LNG,,cancelled,"Louisiana,United States",LNG export terminal,POINT (-90.19444 29.10583)
2,T020601,https://www.gem.wiki/Alaska_LNG_Terminal,Alaska LNG Terminal,T1,proposed,United States,Alaska Gasline Development Corporation [100.00%],State of Alaska [100.00%],6.7,2030,...,GGIT - export,,,293944.41,6.7 mtpa of LNG,,proposed-plus,"Alaska,United States",LNG export terminal,POINT (-151.36505 60.65996)
3,T020602,https://www.gem.wiki/Alaska_LNG_Terminal,Alaska LNG Terminal,T2,proposed,United States,Alaska Gasline Development Corporation [100.00%],State of Alaska [100.00%],6.7,2030,...,GGIT - export,,,293944.41,6.7 mtpa of LNG,,proposed-plus,"Alaska,United States",LNG export terminal,POINT (-151.36505 60.65996)
4,T020603,https://www.gem.wiki/Alaska_LNG_Terminal,Alaska LNG Terminal,T3,proposed,United States,Alaska Gasline Development Corporation [100.00%],State of Alaska [100.00%],6.7,2030,...,GGIT - export,,,293944.41,6.7 mtpa of LNG,,proposed-plus,"Alaska,United States",LNG export terminal,POINT (-151.36505 60.65996)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4353,P6020,https://www.gem.wiki/Strait_West_Natural_Gas_P...,Strait West Natural Gas Pipeline Network Phase II,West-East Gas Pipeline 3 East Section (Ji'an–F...,operating,China,"国家石油天然气管网集团有限公司(60.0002%) , 福建省投资开发集团有限责任公司(39...","国家石油天然气管网集团有限公司(60.0002%) , 福建省投资开发集团有限责任公司(39...",15,2016,...,GGIT,Fujian,Asia,540000.00,15.0 bcm/y of gas,,operating,"Jiangxi,China",gas pipeline,"LINESTRING (114.97011 27.10678, 117.64938 24.5..."
4354,P6031,https://www.gem.wiki/Yunnan_Gas_Pipeline_Netwo...,Yunnan Gas Pipeline Network,Honghe Branch,construction,China,云南中石油昆仑燃气有限公司 [100.00%],云南中石油昆仑燃气有限公司 [100.00%],0.32,,...,GGIT,Yunnan,Asia,11520.00,0.32 bcm/y of gas,,construction-plus,"Yunnan,China",gas pipeline,"LINESTRING (102.55050 24.32886, 102.74441 24.1..."
4355,P6040,https://www.gem.wiki/Sino-Myanmar_Gas_Pipeline...,Sino-Myanmar Gas Pipeline,Chuxiong-Panzhihua Gas Pipeline,operating,China,中国石油天然气股份有限公司 [100.00%],中国石油天然气股份有限公司 [100.00%],2,2019,...,GGIT,Sichuan,Asia,72000.00,2.0 bcm/y of gas,,operating,"Yunnan,China",gas pipeline,"LINESTRING (101.53877 25.03119, 101.67954 26.5..."
4356,P6041,https://www.gem.wiki/Yunnan_Gas_Pipeline_Netwo...,Yunnan Gas Pipeline Network,Xundian-Songming-Airport Branch Gas Pipeline,cancelled,China,云南能投滇中天然气产业发展有限公司 [100.00%],云南能投滇中天然气产业发展有限公司 [100.00%],0.42,,...,GGIT,Yunnan,Asia,15120.00,0.42 bcm/y of gas,,cancelled,"Yunnan,China",gas pipeline,"LINESTRING (103.11865 25.44484, 102.91842 25.1..."


In [145]:
if single_map_file:
    discrepancies = {}
    tracker_to_test = 'INSERT' # THIS HAS TO BE THE ACRONYM IN ALL CONFIGS WHEN DOING THE RENAMING
    source_df = 'TO BE DECLARED BASED ON TRACKER TO TEST'
    map_df = 'TO BE DECLARED ABOVE'
    
    renaming_sel = renaming_cols_dict[tracker_to_test]
    df = source_df.rename(columns=renaming_sel)   
      
    print("-" * 40)
    print(tracker_to_test) # source tracker 


    print(f'Source Data {tracker_to_test}: {len(source_df)}')
    print(f'Map Data: {len(map_df)}')

    df_missing = df[~df['id'].isin(map_df['id'])]
    df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
    
    df_missing = df[~df['id'].isin(map_df['id'])]
    df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
    
    
    print('Map Discrepancy Count')
    print(len(df_missing))
    print('Reverse map discrepancy Count')
    print(len(df_missing_reverse))
    # print(df_missing_reverse)
    if len(df_missing) > 0:
        print(df_missing)
        discrepancies[tracker] = df_missing
    if len(df_missing_reverse) > 0:
        print(df_missing_reverse)
        discrepancies[f'{tracker}-reverse missing'] = df_missing_reverse
            


In [160]:
## for ggit since it has lng and terminals

if map_to_test == 'ggit':
    list_of_trackers = [(ggit_lng, 'GGIT-lng'), (ggit, 'GGIT')]
    discrepancies = {}
        
    for source_tracker_tup in list_of_trackers:

        print("-" * 40)
        print(source_tracker_tup[1])
        tracker = source_tracker_tup[1]
        df = source_tracker_tup[0]
        # print(df)

        renaming_sel = renaming_cols_dict[tracker]
        df = df.rename(columns=renaming_sel) 

        map_df = final_map # TO BE DECLARED
        
        # df = df[df['areas'].isin(needed_geo)] # TODO fix for this because it excludes when there is multiple countries and shouldn't OG0013150 for africa
        print(f'Source Data {tracker}: {len(df)}')
        print(f'Map Data ggit: {len(map_df)}')

        df_missing = df[~df['id'].isin(map_df['id'])]
        df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
        
        df_missing = df[~df['id'].isin(map_df['id'])]
        df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
        
        
        print('Map Discrepancy Count')
        print(len(df_missing))
        print('Reverse map discrepancy Count')
        print(len(df_missing_reverse))
        # print(df_missing_reverse)
        if len(df_missing) > 0:
            print(df_missing)
            
            print(df_missing.columns)
            df_missing.to_csv(f'{testing_errors_path}errors{tracker}.csv')
            discrepancies[tracker] = df_missing
        if len(df_missing_reverse) > 0:
            print(df_missing_reverse)
            discrepancies[f'{tracker}-reverse missing'] = df_missing_reverse
            



----------------------------------------
GGIT-lng
Source Data GGIT-lng: 1295
Map Data ggit: 4358
Map Discrepancy Count
4
Reverse map discrepancy Count
3067
     TerminalID ProjectID       id  \
976       T0823         0  T082300   
1032      T0889         0  T088900   
1105      T0971         0  T097100   
1289      T1174         0  T117400   

                                                    url  \
976   https://www.gem.wiki/New_Fortress_Angola_LNG_T...   
1032  https://www.gem.wiki/Singapore_Offshore_LNG_Te...   
1105  https://www.gem.wiki/SK_Group_Sri_Lanka_LNG_Te...   
1289  https://www.gem.wiki/Golar-Pan_American_FLNG_T...   

                                  name unit_name FacilityType     status  \
976   New Fortress Angola LNG Terminal                 Import  Cancelled   
1032   Singapore Offshore LNG Terminal                 Import  Cancelled   
1105   SK Group Sri Lanka LNG Terminal                 Import    Shelved   
1289  Golar-Pan American FLNG Terminal               

In [98]:

if tracker in gas_only_maps:
    if filter_country:
        # check goit, ggit, ggitlng, goget
        # discrepancy = [(goit,'GOIT'), (ggit, 'GGIT'), (goget,'GOGET'), (ggit_lng,'GGIT-lng')]
        # list_of_trackers = [(coal_terminals, 'GCTT'), (coal_mines, 'GCMT'), (gcpt,'GCPT'), (gogpt, 'GOGPT'), (hydro, 'GHPT'), (wind,'GWPT'), (bio,'GBPT'),
        #                     (solar, 'GSPT'), (geo, 'GGPT'), (nuclear, 'GNPT'), (goget_formatted, 'GOGET'), (ggit_lng, 'GGIT-lng'), (ggit, 'GGIT'), (goit, 'GOIT')]

        # # this one tests against actual data release for goget not the formatted map file, keeping goget_formatted because we use that for regional maps
        # list_of_trackers_dd = [(coal_terminals, 'GCTT Coal Terminals'), (coal_mines, 'GCMT Coal Mines'), (gcpt,'GCPT Coal Plants'), (gogpt, 'GOGPT Oil & GasPlants'), (hydro, 'GHPT Hydropower'), (wind,'GWPT Wind'), (bio,'GBPT Bioenergy'),
        #                     (solar, 'GSPT Solar'), (geo, 'GGPT Geothermal'), (nuclear, 'GNPT Nuclear'), (goget_formatted, 'GOGET Oil & Gas Extraction'), (ggit_lng, 'GGIT-lng LNG Terminals'), 
        #                     (ggit, 'GGIT Gas Pipelines'), (goit, 'GOIT Oil Pipelines')]
        
        # discrepancy = [(goit,'GOIT'), (ggit, 'GGIT'), (goget,'GOGET'), (ggit_lng,'GGIT-lng')]
        list_of_trackers = [(gogpt, 'GOGPT'), (goget_formatted, 'GOGET'), (ggit_lng, 'GGIT-lng'), (ggit, 'GGIT')]
        list_of_trackers_eu = [(gogpt, 'GOGPT'), (goget_formatted, 'GOGET'), (ggit_lng, 'GGIT-lng'), (ggit, 'GGIT'), (ggit_hy, 'GGIT-hy')] # TODO does hy have tracker acro assinged?

        # this one tests against actual data release for goget not the formatted map file, keeping goget_formatted because we use that for regional maps
        list_of_trackers_dd = [(gogpt, 'GOGPT Oil & Gas Plants'), (goget_formatted, 'GOGET Oil & Gas Extraction'), (ggit_lng, 'GGIT-lng LNG Terminals'), 
                            (ggit, 'GGIT Gas Pipelines')]    

        # file_name = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/output_africa_discrepancies_{new_release_date}.xlsx'
        # print(file_name)
        # with pd.ExcelWriter(file_name) as writer:
        if map_to_test == 'asia':
            needed_geo = asia_countries
        # elif map_to_test == 'africa':
        #     needed_geo = africa_countries
        elif map_to_test == 'europe':
            needed_geo = europe_countries
            list_of_trackers = list_of_trackers_eu
        # elif map_to_test == 'latam':
        #     needed_geo = latam_countries
        else:
            needed_geo = ''
        print(f'this is needed geo: {needed_geo}')
        col_country_name = '' # used for hydro we're renaming so not as important we should find the country cols maybe to make this consistent for hydro
        discrepancies = {}
        
        for source_tracker_tup in list_of_trackers:

            print("-" * 40)
            print(source_tracker_tup[1])
            tracker = source_tracker_tup[1]
            df = source_tracker_tup[0]

            renaming_sel = renaming_cols_dict[tracker]
            df = df.rename(columns=renaming_sel) 
            map_df = final_map[final_map['tracker-acro']==tracker]
            if needed_geo == '':
                continue
            else:
                if tracker in tracker_mult_countries: # currently no lists like with regions since goit and ggit created countries a list from start and end countries
                    # if any of the countries in the country column list is in the needed geo list then keep it if none then filter out
                    for sep in ',;-':
                        # I want to break up any multiple countries into a list of countries
                        # then check if any of those are in the needed_geo list
                        df['country_to_check'] = df['areas'].str.strip().str.split(sep) 

                    # filter the df on the country column to see if any of the countries in that list is in the needed geo
                    df = df[df['country_to_check'].apply(lambda x: check_list(x, needed_geo))]
                    # print(tracker)
                    # print(maptype)
                    # print(filtered_df)
                    # input(f'Check length for tracker ggit goit after filter')
                    df = df.drop(columns=['country_to_check'])

                else:
                    # # TODO now that we are only filtering on country for latam we can probalby remove this if else
                    if isinstance(col_country_name, list): # hydro more than one
                        # print(col_country_name)
                        df['country_to_check'] = [[] for _ in range(len(df))]

                        for col in col_country_name:
                            # TEST THIS
                            # filtered_df['country_to_check'] = filtered_df[col].str.strip()
                            df['country_to_check'] = df.apply(lambda row: row['country_to_check'] + [row[col].strip()], axis=1)

                        df = df[df['country_to_check'].apply(lambda x: check_list(x, needed_geo))]
                        df = df.drop(columns=['country_to_check'])

                    else:
                        df['country_to_check'] = df['areas'].str.strip()

                        # need this by country because not a GEM region, problematic since goget uses region and numbers when filtered by country in the gem region do not match 636 versus 578 for africa
                        df = df[df['country_to_check'].isin(needed_geo)]
                    
                        df = df.drop(columns=['country_to_check'])
            print(f'len after geo filter {tracker} {len(df)}')  
            
            if tracker == 'GOGPT':
                # filter out oil only fuel # Alcúdia power station
                drop_row = []
                for row in df.index:
                    fuel_cat_list = df.loc[row, 'fuel'].split(',')
                    new_fuel_cat_list = []
                    for fuel in fuel_cat_list:
                        fuel = fuel.split(':')[0]
                        new_fuel_cat_list.append(fuel)
                    
                    # for Alcudia does not contain gas, or only contains fossil liquids
                    
                    # fossil liquids: diesel, fossil gas: natural ga...      37.5  operating   
                    if len(new_fuel_cat_list) > 1:
                        if new_fuel_cat_list.count('fossil liquids') == len(new_fuel_cat_list):
                                drop_row.append(row)
    
                    elif new_fuel_cat_list == ['fossil liquids']:
                        drop_row.append(row)
                            
                # drop all rows from df that are goget and not in the gas list ids 
                df.drop(drop_row, inplace=True)  
                print(f'len after gas only filter {tracker} {len(df)}')  
            elif tracker == 'GOGET':
                goget_orig_file = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/Global Oil and Gas Extraction Tracker - 2024-03-08_1205 DATA TEAM COPY.xlsx'

                # filter out oil
                list_ids = handle_goget_gas_only_workaround(goget_orig_file)
                # print(len(ndf)) # 3095 will be less because not all trackers
                # filter = (df['tracker-acro']=='GOGET') & (df['prod-gas']=='') #2788
                # filter = df['id'] in list_ids #2788
                # df = df[(df['tracker-acro']=='GOGET') & (df['id'] in list_ids)]
                drop_row = []
                for row in df.index:
                    # if df.loc[row, 'tracker-acro'] == 'GOGET':
                    if df.loc[row, 'id'] not in list_ids:
                        drop_row.append(row)
                # drop all rows from df that are goget and not in the gas list ids 
                df.drop(drop_row, inplace=True)     
                print(f'len after gas only filter {tracker} {len(df)}')  
                

            # df = df[df['areas'].isin(needed_geo)] # TODO fix for this because it excludes when there is multiple countries and shouldn't OG0013150 for africa
            print(f'Source Data {tracker}: {len(df)}')
            print(f'Map Data {tracker}: {len(map_df)}')
            df_missing = df[~df['id'].isin(map_df['id'])]
            df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
            
            print('Map Discrepancy Count')
            print(len(df_missing))
            print('Reverse map discrepancy Count')
            print(len(df_missing_reverse))
            # print(df_missing_reverse)
            if len(df_missing) > 0:
                print(df_missing)
                discrepancies[tracker] = df_missing
            if len(df_missing_reverse) > 0:
                print(df_missing_reverse)
                discrepancies[f'{tracker}-reverse missing'] = df_missing_reverse
        
        for source_tracker_tup in list_of_trackers_dd:

            print("-" * 40)
            tracker_acro = source_tracker_tup[1].split(' ')[0]
            tracker_official = (' ').join(source_tracker_tup[1].split(' ')[1:]) 
            print(tracker_acro)
            print(tracker_official)

            df = source_tracker_tup[0]

            renaming_sel = renaming_cols_dict[tracker_acro]
            df = df.rename(columns=renaming_sel) 
            dd_df = final_data_dict[tracker_official].rename(columns=renaming_sel) 

            df = df[df['areas'].isin(needed_geo)]
            if tracker_acro == 'GOGPT':
                # filter out oil only fuel # Alcúdia power station
                drop_row = []
                for row in df.index:
                    fuel_cat_list = df.loc[row, 'fuel'].split(',')
                    new_fuel_cat_list = []
                    for fuel in fuel_cat_list:
                        fuel = fuel.split(':')[0]
                        new_fuel_cat_list.append(fuel)
                    
                    # for Alcudia does not contain gas, or only contains fossil liquids
                    
                    # fossil liquids: diesel, fossil gas: natural ga...      37.5  operating   
                    if len(new_fuel_cat_list) > 1:
                        if new_fuel_cat_list.count('fossil liquids') == len(new_fuel_cat_list):
                                drop_row.append(row)
    
                    elif new_fuel_cat_list == ['fossil liquids']:
                        drop_row.append(row)
                            
                # drop all rows from df that are goget and not in the gas list ids 
                df.drop(drop_row, inplace=True)  
                print(f'len after gas only filter {tracker} {len(df)}')  
            elif tracker_acro == 'GOGET':
                goget_orig_file = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/Global Oil and Gas Extraction Tracker - 2024-03-08_1205 DATA TEAM COPY.xlsx'

                # filter out oil
                list_ids = handle_goget_gas_only_workaround(goget_orig_file)
                # print(len(ndf)) # 3095 will be less because not all trackers
                # filter = (df['tracker-acro']=='GOGET') & (df['prod-gas']=='') #2788
                # filter = df['id'] in list_ids #2788
                # df = df[(df['tracker-acro']=='GOGET') & (df['id'] in list_ids)]
                drop_row = []
                for row in df.index:
                    # if df.loc[row, 'tracker-acro'] == 'GOGET':
                    if df.loc[row, 'id'] not in list_ids:
                        drop_row.append(row)
                # drop all rows from df that are goget and not in the gas list ids 
                df.drop(drop_row, inplace=True)     
                print(f'len after gas only filter {tracker} {len(df)}')  
            

        
            df_missing_dd = df[~df['id'].isin(dd_df['id'])]
            df_missing_dd_reverse = dd_df[~dd_df['id'].isin(df['id'])]

            print(f'Source Data {tracker_acro}: {len(df)}')
            print(f'Data Download Data {tracker_acro}: {len(dd_df)}')
            print('DD Discrepancy Count')
            print(len(df_missing_dd))
            if len(df_missing_dd) > 0:
                discrepancies[tracker_acro] = df_missing_dd
                print(df_missing_dd)
            if len(df_missing_reverse) > 0:
                print(df_missing_reverse)
                discrepancies[f'{tracker}-reverse missing'] = df_missing_reverse



this is needed geo: ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Albania', 'Andorra', 'Belarus', 'Bosnia and Herzegovina', 'Holy See', 'Iceland', 'Liechtenstein', 'Moldova', 'Monaco', 'Montenegro', 'North Macedonia', 'Norway', 'San Marino', 'Serbia', 'Switzerland', 'Türkiye', 'Ukraine', 'United Kingdom', 'Israel']
----------------------------------------
GOGPT
len after europe filter GOGPT 2049
len after gas only filter GOGPT 1839
Source Data GOGPT: 1839
Map Data GOGPT: 1839
Map Discrepancy Count
0
Reverse map discrepancy Count
0
----------------------------------------
GOGET
len after europe filter GOGET 950
7101
len after gas only filter GOGET 927
Source Data GOGET: 927
Map Data GOGET: 927
Map Discrepancy Count
0
Reverse map d

KeyError: 'Oil & Gas Plants'

CHECK HERE

In [63]:
FAIL HERE

SyntaxError: invalid syntax (36144179.py, line 1)

In [14]:
## For all multi maps not in gas only 
## So LATAM and Africa


if map_to_test not in gas_only_maps:
    # check goit, ggit, ggitlng, goget
    # discrepancy = [(goit,'GOIT'), (ggit, 'GGIT'), (goget,'GOGET'), (ggit_lng,'GGIT-lng')]
    list_of_trackers = [(coal_terminals, 'GCTT'), (coal_mines, 'GCMT'), (gcpt,'GCPT'), (gogpt, 'GOGPT'), (hydro, 'GHPT'), (wind,'GWPT'), (bio,'GBPT'),
                        (solar, 'GSPT'), (geo, 'GGPT'), (nuclear, 'GNPT'), (goget_formatted, 'GOGET'), (ggit_lng, 'GGIT-lng'), (ggit, 'GGIT'), (goit, 'GOIT')]

    # this one tests against actual data release for goget not the formatted map file
    list_of_trackers_dd = [(coal_terminals, 'GCTT Coal Terminals'), (coal_mines, 'GCMT Coal Mines'), (gcpt,'GCPT Coal Plants'), (gogpt, 'GOGPT Oil & GasPlants'), (hydro, 'GHPT Hydropower'), (wind,'GWPT Wind'), (bio,'GBPT Bioenergy'),
                        (solar, 'GSPT Solar'), (geo, 'GGPT Geothermal'), (nuclear, 'GNPT Nuclear'), (goget, 'GOGET Oil & Gas Extraction'), (ggit_lng, 'GGIT-lng LNG Terminals'), 
                        (ggit, 'GGIT Gas Pipelines'), (goit, 'GOIT Oil Pipelines')]

    # file_name = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/output_africa_discrepancies_{new_release_date}.xlsx'
    # print(file_name)
    # with pd.ExcelWriter(file_name) as writer:
    needed_geo = f'{map_to_test}_countries'

    discrepancies = {}
    for source_tracker_tup in list_of_trackers:

        print("-" * 40)
        print(source_tracker_tup[1])
        tracker = source_tracker_tup[1]
        df = source_tracker_tup[0]

        renaming_sel = renaming_cols_dict[tracker]
        df = df.rename(columns=renaming_sel) 
        map_df = final_map[final_map['tracker acro']==tracker]
        if tracker == 'GOGET':
            df = df[df['areas'].isin(needed_geo)] # TODO fix for this because it excludes when there is multiple countries and shouldn't OG0013150 for africa
            print(f'Source Data {tracker}: {len(df)}')
            print(f'Map Data {tracker}: {len(map_df)}')
            df_missing = df[~df['id'].isin(map_df['id'])]
            df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
            
            print('Map Discrepancy Count')
            print(len(df_missing))
            print(df_missing_reverse)
            if len(df_missing) > 0:
                # print(df_missing)
                discrepancies[tracker] = df_missing
                
        elif tracker == 'GCMT':
            df = df[df['areas'].isin(needed_geo)] # TODO fix for this because it excludes when there is multiple countries and shouldn't OG0013150 for africa
            print(f'Source Data {tracker}: {len(df)}')
            print(f'Map Data {tracker}: {len(map_df)}')
            df_missing = df[~df['id'].isin(map_df['id'])]
            df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
            
            print('Map Discrepancy Count')
            print(len(df_missing))
            print(df_missing_reverse)
            if len(df_missing) > 0:
                print(df_missing) 
                print(f'This is region for missing: {df_missing["region"]}')
            
            # filter for no status
            print('mapdf with no status gcmt:')
            map_df['status'] = map_df['status'].fillna('')
            map_df_no = map_df[map_df['status']=='']
            print(map_df_no)
            print('source df with no status gcmt:')   
            df['status'] = df['status'].fillna('')
            df_no = df[df['status']==''] 
            print(df_no)   
        else:
            df = df[df['region']==f'{map_to_test.title()}']
            print(f'Source Data {tracker}: {len(df)}')
            print(f'Map Data {tracker}: {len(map_df)}')

            df_missing = df[~df['id'].isin(map_df['id'])]
            df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]

            if len(df_missing) > 0:
                print(f'df missing {df_missing}')
                if 'geometry' in df_missing.columns:
                    df_investigation_7 = df_missing[df_missing['geometry']!= '']
                    # print(df_investigation_7['geometry'])
                discrepancies[tracker] = df_missing
            print('Map Discrepancy Count')
            print(len(df_missing))
            # if len(df_missing_reverse) >0:
                # print(f'reverse {df_missing_reverse}')

            
    for source_tracker_tup in list_of_trackers_dd:

        print("-" * 40)
        tracker_acro = source_tracker_tup[1].split(' ')[0]
        tracker_official = source_tracker_tup[1].split(' ')[1:]
        print(tracker_acro)
        print(tracker_official)

        df = source_tracker_tup[0]

        renaming_sel = renaming_cols_dict[tracker_acro]
        print(renaming_sel)
        print(type(renaming_sel)) # dict but getting error of list
        df = df.rename(columns=renaming_sel) 
        dd_df = final_data_dict[tracker_official].rename(columns=renaming_sel) 

        if tracker_acro == 'GOGET':
            df = df[df['areas'].isin(needed_geo)]
            df_missing_dd = df[~df['id'].isin(dd_df['id'])]
            df_missing_dd_reverse = dd_df[~dd_df['id'].isin(df['id'])]

            print(f'Source Data {tracker_acro}: {len(df)}')
            print(f'Data Download Data {tracker_acro}: {len(dd_df)}')
            print('DD Discrepancy Count')
            print(len(df_missing_dd))
            if len(df_missing_dd) > 0:
                # print(df_missing_dd['region'])
                discrepancies[tracker_acro] = df_missing_dd

        else:
            df = df[df['region']=={map_to_test.title}]
            print(f'Source Data {tracker_acro}: {len(df)}')
            print(f'Data Download Data {tracker_acro}: {len(dd_df)}')
            df_missing_dd = df[~df['id'].isin(dd_df['id'])]
            df_missing_dd_reverse = dd_df[~dd_df['id'].isin(df['id'])]

            print('DD Discrepancy Count')
            print(len(df_missing_dd))
            if len(df_missing_dd) > 0:
                # print(df_missing_dd['region'])
                discrepancies[tracker_acro] = df_missing_dd
            # print(df_missing_dd_reverse)



In [15]:
# for GIPT 
# test that the James file is identical to the source data
# test that my file is identical to both
list_source_trackers_gipt = [(gcpt,'GCPT'), (gogpt, 'GOGPT'), (hydro, 'GHPT'), (wind,'GWPT'), (bio,'GBPT'),
                    (solar, 'GSPT'), (geo, 'GGPT'), (nuclear, 'GNPT')]

print(len(gipt_james_version))
print(gipt_james_version['Country/area'].value_counts())
print(gipt_james_version['Status'].value_counts())
print(gipt_james_version['Region'].value_counts())
print(gipt_james_version['Capacity (MW)'].sum())
print(gipt_james_version['Type'].value_counts())

for source_tracker_tup in list_source_trackers_gipt:

    print("-" * 40)
    print(source_tracker_tup[1])
    tracker = source_tracker_tup[1]
    df = source_tracker_tup[0]

    renaming_sel = renaming_cols_dict[tracker]
    df = df.rename(columns=renaming_sel) 
    print(len(df))
    

116466
Country/area
China             31070
United States     13857
Germany            9292
Brazil             5406
Spain              5297
                  ...  
Andorra               1
Tonga                 1
American Samoa        1
Guernsey              1
Jersey                1
Name: count, Length: 207, dtype: int64
Status
operating           74330
pre-construction    16970
cancelled            7200
construction         5604
announced            5083
retired              3803
shelved              2910
mothballed            566
Name: count, dtype: int64
Region
Asia        47947
Europe      40055
Americas    24345
Africa       2818
Oceania      1301
Name: count, dtype: int64
18568964.46
Type
solar         51968
wind          29930
coal          14004
oil/gas       12436
hydropower     4203
bioenergy      1807
nuclear        1541
geothermal      577
Name: count, dtype: int64
----------------------------------------
GCPT
14004
----------------------------------------
GOGPT
12436
-----